## Rankability Predicting Sensitivity
### March Madness Dataset

Look for new features:</br>
Existing features:</br>
Diversity of P metrics</br>
Graph measures of tournament matrix as Lapacian</br>
Eigenvalues of tournament matrix</br>

Datasets:</br>

Lichess:</br> API: https://berserk.readthedocs.io/en/master/ </br> Look for tournaments, grab games played in that time frame and create D matricies</br> API is pretty simple we just need to build a scraping script.</br>

Sumo: Data: https://data.world/cervus/sumo-results </br> It's literally just CSVs, so grab to PANDAS and build D from columns Bad news: Have to make an account to download data :( /s

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import copy
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import skew

In [ ]:
import sys
# We need to include the path to pyrankability. This could be different for Tim, but altneratively he could point to your copy
sys.path.insert(0,"/disk/home/amy/rankability_toolbox_dev")
import pyrankability

In [ ]:
sys.path.insert(0,"/disk/home/amy/sensitivity_study/src")
from sensitivity_tests import *
from utilities import *
from base import *

In [ ]:
games={}
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"]
for year in years:
    games[year] = read_data('../data/%steams.txt'%year,'../data/%sgames.txt'%year,'../data/%sMadnessTeams.txt'%year)
games[year]

In [ ]:
# Note to future self: Parameters from FODS paper but might need to be optimized
direct_thres = 2
spread_thres = 2
weight_indirect = 0.5
Ds = {}
fracs = [0.5, 0.75, 1.] # 25% of total data added per step
#fracs = [0.64, 0.8, 1.] # 25% of current data added per step
pairs = list(zip(fracs[:-1], fracs[1:]))
pair_to_predict = pairs[0]

for year in games.keys():
    print(year)
    Ds[year] = {}
    madness_teams = np.unique(list(games[year].team1_name.loc[games[year].team1_madness == 1]) + list(games[year].team2_name.loc[games[year].team2_madness == 1]))
    game_list = list(games[year].index)
    
    game_df = pd.DataFrame({"team1_name":games[year]['team1_name'],
                            "team1_score":games[year]['points1'],
                            "team1_H_A_N": games[year]['H_A_N1'],
                            "team2_name":games[year]['team2_name'],
                            "team2_score":games[year]['points2'],
                            "team2_H_A_N": games[year]['H_A_N1'],
                            "date": games[year]['date']
                           }).sort_values(by='date').drop('date',axis=1)
    for frac in fracs:
        upper = int(len(game_df)*frac)
        game_df_sample = game_df.iloc[:upper,:]
        map_func = lambda linked: support_map_vectorized_direct_indirect_weighted(linked,direct_thres=direct_thres,spread_thres=spread_thres,weight_indirect=weight_indirect)
        Ds[year][frac] = pyrankability.construct.V_count_vectorized(game_df_sample,map_func).loc[madness_teams,madness_teams]

In [ ]:
rankings = {}
taus = {}
results = pd.DataFrame(columns=pairs+["Year"]).set_index("Year")

for year in games.keys():
    rankings[year] = []
    taus[year] = {}
    data = []
    for i in range(len(pairs)):
        pair = pairs[i]
        D1 = Ds[year][pair[0]]
        D2 = Ds[year][pair[1]]
        ranking1 = MasseyRankingAlgorithm().rank(D1.fillna(0).values)
        ranking2 = MasseyRankingAlgorithm().rank(D2.fillna(0).values)
        rankings[year].append((ranking1,ranking2))
        ranking1, ranking2 = rankings[year][i]
        taus[year][pair] = kendall_tau(ranking1,ranking2)
        data.append(taus[year][pair])
    results = results.append(pd.Series(data,index=results.columns,name=year))

In [ ]:
taus

In [ ]:
results

In [ ]:
# Note: what do the contents of this matrix mean??
# Ds['2018'][1.0]

In [ ]:
df_details = []
def get_rankability_results(n_restarts=250):
    df_ks = []
    df_years = []
    df_fracs = []
    df_p_stats = {}
    for year in games.keys():
        print(year)
        for pair in pairs:
            D = Ds[year][pair[0]].fillna(0)
            #C = pyrankability.construct.C_count(D,0)
            k,details = pyrankability.rank.solve(D,method='lop', num_random_restarts=n_restarts, lazy=False, cont=True)
            p_stats = get_P_stats(details["P"])
            for name, val in p_stats.items():
                if name not in df_p_stats:
                    df_p_stats[name] = []
                df_p_stats[name].append(val)
            df_ks.append(k)
            df_years.append(year)
            df_fracs.append(pair[0])
            df_details.append(details)

    results_temp = {"k":df_ks,"Year":df_years,"Fraction":df_fracs}
    for key, val in df_p_stats.items():
        if key in results_temp:
            raise ValueError("Duplicate column name! Fix collision before moving on!")
        results_temp[key] = val

    return pd.DataFrame(results_temp)

In [ ]:
rankability_results = get_rankability_results()
rankability_results

In [ ]:
for_corr = rankability_results.loc[rankability_results.Fraction==pair_to_predict[0]].set_index('Year').join(results)
for_corr

In [ ]:
for_corr.corr()

In [ ]:
pearsonr(for_corr['k'],for_corr[pair_to_predict])

In [ ]:
col_names = [
    'Year',
    '# X* frac',
    'k',
    '# X* frac top 40',
    'kendall_w',
    'p_lowerbound',
    'max_L2_dist',
    'mean_L2_dist',
    'min_tau',
    'mean_tau',
    'Pair'
]

all_score_df = pd.DataFrame(columns=col_names)

c=0
for year in games.keys():
    print(year)
    for pair in pairs:
        V = Ds[year][pair_to_predict[0]]
        rresults = rankability_results.iloc[c,:]
        k = rresults['k']
        details = df_details[c]
        x = pd.DataFrame(details['x'],index=V.index,columns=V.columns)
        r = x.sum(axis=0)
        order = np.argsort(r)
        xstar = x.iloc[order,:].iloc[:,order]
        xstar.loc[:,:] = pyrankability.common.threshold_x(xstar.values)
        inxs = np.triu_indices(len(xstar),k=1)
        xstar_upper = xstar.values[inxs[0],inxs[1]]
        nfrac_upper = sum((xstar_upper > 0) & (xstar_upper < 1))
        flat_frac = ((xstar > 0) & (xstar < 1)).sum(axis=0)
        nfrac_top_40 = flat_frac.iloc[:40].sum()
        entry_data = [
            year,
            nfrac_upper*2,
            k,
            nfrac_top_40,
            rresults["kendall_w"],
            rresults["p_lowerbound"],
            rresults["max_L2_dist"],
            rresults["mean_L2_dist"],
            rresults["min_tau"],
            rresults["mean_tau"],
            pair
        ]
        entry = pd.Series(entry_data,col_names,name=c)
        c+=1
        all_score_df = all_score_df.append(entry)
all_score_df.set_index("Year",inplace=True)

In [ ]:
all_score_df = all_score_df.loc[all_score_df.Pair == pair_to_predict].drop('Pair',axis=1).join(results)

In [ ]:
all_score_df

In [ ]:
# TODO: Save all_score_df for easy reuse
# TODO: add function to reload all_score_df from saved file
all_score_df.to_csv("all_score_df.csv")

# all_score_df = pd.read_csv("all_score_df.csv")

In [ ]:
for col in all_score_df.columns:
    if col not in pairs:
        all_score_df.plot.scatter(col, pair_to_predict, title="Final Sensitivity vs " + col)

In [ ]:
import sklearn.linear_model as skl_lm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import GridSearchCV

all_feature_cols = [c for c in all_score_df.columns if c not in pairs]

def evaluate(df,pred_col=pair_to_predict,feature_cols=all_feature_cols,model=DummyRegressor(),param_grid={}):
    # fill in evaluat
    loo = LeaveOneOut()
    y = df[pred_col]
    X = df[feature_cols]
    
    grid = GridSearchCV(model,param_grid,refit=True,verbose=2,n_jobs=-1)
    scores = cross_val_score(grid, X, y, scoring="neg_mean_absolute_error", cv=loo, n_jobs=1)
    return pd.Series([len(scores),np.mean(np.abs(scores)),np.std(scores)],index=["Folds","MAE","STD"])

In [ ]:
evaluate(all_score_df)

In [ ]:
evaluate(all_score_df,model=SVR(gamma='scale'),param_grid = {'C': [0.1,1,10,100], 'epsilon': [0.1,0.5,1],'kernel': ['linear', 'rbf']})